In [75]:
#IMPORTS
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV #splits up your test set in eqally sized parts, uses one part as test data and the rest as training data. So it optimizes as many classifiers as parts you split your data into.

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier


In [76]:
#COUNT UNIQUE USERS

df = pd.read_csv('USER-FEATURE-DATA.csv')

current_row=0
df_sample = pd.DataFrame([],columns=df.columns)
counter = 0
for i in np.unique(df['user1']):
      counter +=1

print(counter) 

20


In [77]:
#LOAD DATA

df = pd.read_csv('NEW_SAMPLES.csv')

current_row=0
df_sample = pd.DataFrame([],columns=df.columns)

samples = []

N = 10

for i in np.unique(df['user1']):
      try:
            new_sample=df[df['user1']==i].sample(n=N)
            samples.append(new_sample)
            
      except:
            pass
df_sample = []

for i in range(0, N):
      for j in range(0, len(samples)):
            data_slice = samples[j].iloc[i:i+1, :]
            df_sample.append(data_slice)
#print(df_sample[2].iloc[0:, 0])

df_sample = pd.concat(df_sample, axis=0, ignore_index=True)


y = df_sample.iloc[0:, 0]
X = df_sample.iloc[0:, 1:]
print(y.head(42))

0      user1
1     user19
2      user2
3     user27
4     user28
5     user29
6      user3
7     user33
8     user35
9     user36
10    user39
11     user4
12    user40
13    user43
14    user44
15    user46
16    user47
17    user48
18    user49
19     user7
20     user1
21    user19
22     user2
23    user27
24    user28
25    user29
26     user3
27    user33
28    user35
29    user36
30    user39
31     user4
32    user40
33    user43
34    user44
35    user46
36    user47
37    user48
38    user49
39     user7
40     user1
41    user19
Name: user1, dtype: object


In [102]:
#K NEAREST NEIGHBOURS
clf = KNeighborsClassifier(n_neighbors=1,algorithm='auto', metric='euclidean')

kf = KFold(n_splits=5)

for train_index, test_index in kf.split(X):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]

    clf.fit(X_train, y_train)
    print('SCORE:', clf.score(X_test, y_test))

SCORE: 0.75
SCORE: 0.675
SCORE: 0.8
SCORE: 0.75
SCORE: 0.75


In [127]:
#RANDOM FORESTS CLASSIFIER
import math
clf = RandomForestClassifier(n_estimators=100, random_state=0)

kf = KFold(n_splits=5)

for train_index, test_index in kf.split(X):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]

    clf.fit(X_train, y_train)
    print('SCORE:', clf.score(X_test, y_test))


SCORE: 0.825
SCORE: 0.65
SCORE: 0.825
SCORE: 0.8
SCORE: 0.775


In [ ]:
#TRAINING

kf = KFold(n_splits=5)

ATTRIBUTES = len(X.columns)
print('n attrb =', ATTRIBUTES)
USERS = y.nunique()
print('n users =', USERS)
neurons = (ATTRIBUTES + USERS) // 2 
clf = MLPClassifier(hidden_layer_sizes=(neurons,), solver='lbfgs', momentum=0.2, learning_rate_init=0.3, random_state=0)

clf.max_iter = 10000

clf.n_layers_ = 5

#got rid of:
#hidden_layer_sizes=(1,) (scores all zeroes)
#  l
#
# look into RandomSearchCV
param_grid = [
        {
            'activation' : ['identity', 'logistic', 'tanh', 'relu'],
            'solver' : ['lbfgs', 'sgd'],
            'hidden_layer_sizes': [
              (neurons-10,), (neurons-5,), (neurons,), (neurons+5,), (neurons+10,)
             ],
             'momentum': [0.1,0.2,0.3],
             'learning_rate_init':[0.2,0.3,0.4]
        }
       ]

GRID_SEARCH_TEST = False 

for train_index, test_index in kf.split(X):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]

    if GRID_SEARCH_TEST:
        clf = GridSearchCV(MLPClassifier(), param_grid, cv=3, scoring='accuracy')
        clf.fit(X_train, y_train)
        print('params:', clf.best_params_)
    else:
      clf.n_layers_ = 5
      clf.fit(X_train, y_train)

      print('SCORE:', clf.score(X_test, y_test))
    
    
    #EXTRA INFO:
     

      #print('current loss computed with the loss function: ',clf.loss_)
      #print('coefs: ', clf.coefs_)
      #print('intercepts: ',clf.intercepts_)
      #print(' number of iterations the solver: ', clf.n_iter_)
      #print('num of layers: ', clf.n_layers_)
      #print('Num of o/p: ', clf.n_outputs_) 
      
    



n attrb = 102
n users = 20
SCORE: 0.725
SCORE: 0.8
SCORE: 0.75
SCORE: 0.9
SCORE: 0.825
